In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
df = pd.read_csv("/content/melb_data.csv")

In [ ]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [ ]:
df = df.dropna(subset=["Price"])

In [ ]:
df = df.dropna(subset=["Price"])

In [ ]:
X = df.select_dtypes(include=[np.number]).drop(columns=["Price"]).fillna(0)
y = df["Price"]

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
# Scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## pytorch MLP

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
# DataLoader
train_ds = TensorDataset(
    torch.tensor(X_train, dtype=torch.float32),
    torch.tensor(y_train.values, dtype=torch.float32).view(-1,1)
)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)

In [ ]:
# Model
class MLP(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_dim, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x): return self.fc(x)

In [ ]:
model_pt = MLP(X_train.shape[1])
loss_fn = nn.MSELoss()
opt = torch.optim.Adam(model_pt.parameters(), lr=0.001)


In [ ]:
# Train
for epoch in range(10):
    for xb, yb in train_loader:
        pred = model_pt(xb)
        loss = loss_fn(pred, yb)
        opt.zero_grad(); loss.backward(); opt.step()
    print(f"PT Epoch {epoch+1}, Loss {loss.item():.2f}")


PT Epoch 1, Loss 1283598450688.00
PT Epoch 2, Loss 1805987479552.00
PT Epoch 3, Loss 2290306252800.00
PT Epoch 4, Loss 1504329203712.00
PT Epoch 5, Loss 1495579754496.00
PT Epoch 6, Loss 1779996033024.00
PT Epoch 7, Loss 1424776364032.00
PT Epoch 8, Loss 1164748652544.00
PT Epoch 9, Loss 1420248088576.00
PT Epoch 10, Loss 1286479806464.00


In [ ]:
# Evaluate
with torch.no_grad():
    preds_pt = model_pt(torch.tensor(X_test, dtype=torch.float32)).numpy().squeeze()
mse_pt = mean_squared_error(y_test, preds_pt)
mae_pt = mean_absolute_error(y_test, preds_pt)
print("\nPyTorch Results -> MSE:", mse_pt, "MAE:", mae_pt)


PyTorch Results -> MSE: 1375039621886.594 MAE: 1006617.5755838162


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model_tf = keras.Sequential([
    layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation="relu"),
    layers.Dense(1)
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model_tf.compile(optimizer="adam", loss="mse", metrics=["mae"])

history = model_tf.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    verbose=1
)

Epoch 1/10
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 1520831299584.0000 - mae: 1065298.2500 - val_loss: 1593106628608.0000 - val_mae: 1082414.6250
Epoch 2/10
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1566894456832.0000 - mae: 1073120.2500 - val_loss: 1592382455808.0000 - val_mae: 1082128.5000
Epoch 3/10
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1530704822272.0000 - mae: 1063578.3750 - val_loss: 1590142042112.0000 - val_mae: 1081257.5000
Epoch 4/10
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 1620081377280.0000 - mae: 1090450.7500 - val_loss: 1585655578624.0000 - val_mae: 1079520.2500
Epoch 5/10
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1558176464896.0000 - mae: 1069840.6250 - val_loss: 1578274258944.0000 - val_mae: 1076663.5000
Epoch 6/10
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1558217097216.0000 - mae: 1069754.6250 - val_loss: 1567571181568.0000 - val_mae: 1072509.0000
Epoch 7/10
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1589172895744.

In [ ]:
mse_tf, mae_tf = model_tf.evaluate(X_test, y_test, verbose=0)
print("\nTensorFlow Results -> MSE:", mse_tf, "MAE:", mae_tf)


TensorFlow Results -> MSE: 1447436746752.0 MAE: 1034854.375
